In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from matplotlib import font_manager, rc
font_path = "c:\WINDOWS\Fonts\GULIM.TTC"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
plt.rc('font', size=8) 

# 데이터 프레임 읽어오기

In [ ]:
dataframe = pd.read_csv(r'C:\Users\hyssk\AICE_\open\train.csv')
dataframe.drop(columns='ID',inplace=True)
dataframe

# dataframe.info() 통해서 데이터 개수 및 데이터 정보 및 결측치 개수 확인

In [ ]:
dataframe.info()

수치형 데이터 칼럼은 9개, 카테고리형(범주형) 데이터 칼럼 5개

데이터 개수는 각 칼럼 96294개의 관측치 존재

데이터 결측치는 존재하지 않는 걸로 보이지만, 아직 정확하게 알 수 있는건 아니어서 직접 열어서 확인해보는 것이 Best

# dataframe.describe() 통해서 데이터 요약 통계량 확인

In [ ]:
dataframe.describe()

In [ ]:
categorical_columns = dataframe.select_dtypes('object').columns # 범주형 데이터 칼럼
numeric_columns = dataframe.select_dtypes('number').columns # 수치혀 데이터 칼럼

categorical_columns,numeric_columns

In [ ]:
dataframe[numeric_columns].describe() # 수치형 데이터 칼럼의 요약 통게량

Insight

1. 부채 대비 소득 비율 파트에서 Q1 Q2 Q3는 12,18,25 인데, 최대값이 9999이다. (이상치는 아닌지 확인할 필요)

2. 최근 2년간 연체 횟수 max 값도 30도 확인해 볼 필요가 있다.

3. 총연체금액도 위와 같은 명목이다. 평균 54이고, std (표준편차)도 1414인데, 최대치는 75768은 높은 확률로 이상치일 가능성이 있다.

In [ ]:
dataframe[categorical_columns].describe() # 범주형 데이터 칼럼의 요약 통계량

unique는 ex) 대출기간이라는 (인자) 내에서의 수준(level) 또는 treatment(처리) 개수이다.

top은 제일 많이 나오는 수준

freq은 제일 많이 나오는 수준의 빈도수

Insight

1. 대출기간의 수준은 2개 근로기간 16개 주택소유상태 4개 대출목적 12개 대출등급 7개

2. 근로기간의 수준은 16개 너무 많은데 확인할 필요가 있어보임

# Insight 확인

1. 부채 대비 소득 비율 파트에서 Q1 Q2 Q3는 12,18,25 인데, 최대값이 9999이다. (이상치는 아닌지 확인할 필요)

In [ ]:
plt.figure()
plt.scatter(x=dataframe['부채_대비_소득_비율'],y=dataframe['근로기간'])
plt.show()


근로기간 수준이 Unknown인 자료에서 부채_대비_소득_비율이 10000인 값이 관측되었다.

In [ ]:
dataframe[dataframe['부채_대비_소득_비율'] > 8000]

부채 대비 소득 비율이 9999인 사람이 존재한다.

부채 대비 소득 비율이란 (부채 / 연간 소득) 이다.

따라서 4440000 / 0 이어서 9999.0  으로 측정된 듯 하다.

사실 0으로 값을 나눈 다는 것은 정의되지 않지만, 값을 넣어주어야 하기에 9999로 정해서 넣은 것 같다.

따라서 본인은 이 값을 행을 제거시킴으로써 데이터 전처리를 하고자 한다.


In [ ]:
dataframe.drop(index = 12498,inplace=True)

dataframe[dataframe['부채_대비_소득_비율'] > 8000] # 데이터 행 삭제가 잘 되었는지 확인

In [ ]:
plt.figure()
plt.scatter(x=dataframe['부채_대비_소득_비율'],y=dataframe['근로기간'])
plt.show()

시각화로 다시 한번 확인하였다. 그러자 다시 한번 보이는 600 이상의 이상치들이 보이기 시작한다. 

In [ ]:
dataframe[dataframe['부채_대비_소득_비율'] > 600]


나는 어떻게 해야할까? 

해당 2행이 정상적인 데이터면 어떻게하지?

이상치라면 데이터 사이언티스트로서 해결해야할까?

전처리를 어떻게 처리해야하는가?에 대한 고민이 된다.

어떤 기준을 가지고 전처리를 해야하는가..

우선은 삭제를 통해서 전처리해야겠다.

In [ ]:
dataframe.drop(index=dataframe[dataframe['부채_대비_소득_비율'] > 600].index,inplace=True)

plt.figure()
plt.boxplot(dataframe['부채_대비_소득_비율'])
plt.xlabel('부채 대비 소득 비율')
plt.ylabel('값 분포')
plt.show()

가장 큰 3개의 이상치로 의심되는 관측치를 삭제한 뒤 Boxplot을 그려보았다.

그럼에도 불구하고 아직도 부채 대비 소득 비율의 이상치들이 많이 발견되었다.

# 2. 최근 2년간 연체 횟수 max 값도 30도 확인해 볼 필요가 있다.

In [ ]:
dataframe[['최근_2년간_연체_횟수']].describe()

In [ ]:
dataframe[dataframe['최근_2년간_연체_횟수']>0]

print(f'최근 2년간 연체 횟수가 0 초과인 행의 비율은 : {(19900/96291)*100:.2f}% 입니다. ')
print(f'Q1 = 0.0, Q2 = 0.0, Q3 = 0.0, Mean: 0.34 입니다.')

Q1 Q2 Q3 다 0값이고 Mean: 0.34이고 0인 값들이 약 80퍼센트에 가깝고, 0인 값들이 약 20퍼센트에 가깝다.

In [ ]:
plt.figure()
plt.scatter(x = dataframe['최근_2년간_연체_횟수'], y= dataframe['근로기간'])
plt.xlabel('최근 2년간 연체 횟수')
plt.show()

In [ ]:
dataframe.drop(index=57431,inplace=True)
dataframe

In [ ]:
plt.figure()
plt.boxplot(x=dataframe['최근_2년간_연체_횟수'])
plt.show()

이상치가 아닐 수 있다는 생각이 든다. 우리가 80퍼센트의 관측치들이 0이어서 평균이 0에 가깝게 나타나는 것이지,

0이 아닌 값들이 이상치로 판단하는 것은 무리가 있어 보인다. 

따라서 본인은 이러한 이상치를 전처리를 하지않고 그대로 가져가는 것을 택한다.

# 3. 총연체금액도 위와 같은 명목이다. 평균 54이고, std (표준편차)도 1414인데, 최대치는 75768은 높은 확률로 이상치일 가능성이 있다.

In [ ]:
plt.figure()
plt.boxplot(x=dataframe['총연체금액'])
plt.show()

이상치가 아닐 수 있다는 생각이 든다. 위의 내용과 동일하다.

0이 아닌 값들이 이상치로 판단하는 것은 무리가 있어 보인다. 

따라서 본인은 이러한 이상치를 전처리를 하지않고 그대로 가져가는 것을 택한다.

# 시각화를 통한 Insight 도출하기 (시각화)

## 대출기간 | 대출등급

In [ ]:
anova_frame=dataframe[['대출금액','대출기간','대출등급']]
print('대출 기간의 수준:', anova_frame['대출기간'].unique())
print('대출 등급의 수준:', anova_frame['대출등급'].unique())
print(anova_frame)
# 등급과 대출기간 평균이 -대출 금액의 평균의 차이가존재하는가?
fit=smf.ols('대출금액~ 대출기간 + 대출등급 + 대출기간*대출등급',anova_frame).fit()
sm.stats.anova_lm(fit)

대출기간 수준: [36 months / 60 months] 간의 대출금액 평균 차이는 없다라는 가설을 기각

대출등급 수준: ['C' 'B' 'A' 'D' 'F' 'G' 'E'] 간의 대출금액 평균 차이는 없다라는 가설을 기각

교호효과는 없다라는 가설도 기각된다.

즉, 
1. 대출 기간 수준 간의 적어도 대출 금액 평균 차이가 존재
2. 대출 등급 수준 간의 적어도 대출 금액 평균 차이가 존재
3. 대출 등급과 대출 기간 간의 교호효과가 존재한다.


In [ ]:
mean_period_level = anova_frame[['대출기간','대출금액']].groupby('대출기간').mean()
mean_grade_level = anova_frame[['대출등급','대출금액']].groupby('대출등급').mean()

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.bar(list(mean_period_level.index),[15383401.16324157,24223658.65157944])
plt.xlabel('대출기간')
plt.ylabel('평균')
plt.subplot(1,2,2)
plt.bar(list(mean_grade_level.index),[17905794.7770093,16868082.72894472,
                                      17890527.85923754,19534187.82296113,
                                      22321207.67033864,24005174.00204708,24020785.71428571])
plt.xlabel('대출등급')
plt.ylabel('평균')
plt.show()


대출기간 수준: [36 months / 60 months] 간의 대출금액 평균 차이는 없다라는 가설을 기각

대출 기간 수준의 평균 차이가 나므로 가설 기각에 힘이 실린다. 

대출등급 수준: ['C' 'B' 'A' 'D' 'F' 'G' 'E'] 간의 대출금액 평균 차이는 없다라는 가설을 기각

대출금액 간의 평균 차이가 적어도 하나라도 있으므로 가설 기각에 힘이 실린다.

## 대출등급 | 대출기간들의 (대출금액에 대한 std)

In [ ]:
A=anova_frame[['대출기간','대출금액']].groupby('대출기간')
B=anova_frame[['대출등급','대출금액']].groupby('대출등급')
plt.figure()
sns.scatterplot(data=A.std())
plt.axhline(y=A.mean().values[0],color='g',linewidth=1,label='36 months mean')
plt.axhline(y=A.mean().values[1],color='r',linewidth=1,label='60 months mean')
plt.legend()
plt.show()

A.std()

In [ ]:
plt.figure()
sns.scatterplot(data=B.std())
plt.axhline(y=B.mean().values[0],color='g',linewidth=7,label='A')
plt.axhline(y=B.mean().values[1],color='r',linewidth=1,label='B')
plt.axhline(y=B.mean().values[2],color='b',linewidth=3,label='C')
plt.axhline(y=B.mean().values[3],color='k',linewidth=1,label='D')
plt.axhline(y=B.mean().values[4],color='c',linewidth=1,label='E')
plt.axhline(y=B.mean().values[5],color='m',linewidth=1,label='F')
plt.axhline(y=B.mean().values[6],color='y',linewidth=1,label='G')
plt.legend()
plt.show()

## 분산 분석의 기본 전제는 3가지 - 등분산성, 독립성, 정규성이다.

지금 위의 그래프는 등분산이라고는 보기 힘들다. 그래서 자료변환을 진행한 뒤, 다시한번 분산분석을 진행하려 한다.

In [ ]:
# Log 대수변환 

anova_frame['log대출금액'] = np.log(anova_frame['대출금액'])
anova_frame['sqrt대출금액'] = np.sqrt(anova_frame['대출금액'] + 0.375)
anova_frame

In [ ]:
plt.figure()
sns.scatterplot(data=anova_frame[['대출금액','대출기간']].groupby('대출기간').std())
sns.scatterplot(data=anova_frame[['대출금액','대출기간']].groupby('대출기간').std())
plt.show()

# 근로기간 Column의 이상한 점 (1 years와 1 year차이 파악 / 10+ years와 10+years의 차이 파악 / < 1 year와 <1 year 차이 파악와 3 years와 3과의 차이 파악)

즉, 근로기간의 Level에 대한 통일이 전혀 안되어있는 걸로 보인다.

## groupby(범주형 칼럼명).mean() 오류 디버깅

If you got a similar TypeError after a groupby operation (e.g. TypeError: Could not convert ace to numeric), then you probably have pandas>=2.0.

groupby.mean() has numeric_only= argument whose default value was True in the past but since pandas 2.0, its default value is False. An implication is that string columns are not dropped when a statistical method such as mean or std is called on the groupby object (as was done in the past). To solve the issue, pass numeric_only=True.

즉, 2.0 이하 버전에서는 numeric_only = True 가 Default 값이었지만, 2.0 이상 버전에서는 numeric_only (파라미터)가 False 로 변경 되면서 문자열 칼럼은 통계적 방법(평균/표준편차가 호출될 때, drop(떨어져나가지)않아서 발생되는 오류)

numeric_only=True를 사용해서 mean(),std()를 구해야함

In [ ]:
duration = dataframe.groupby('근로기간').mean(numeric_only=True)
duration

In [ ]:
plt.figure(figsize=(12,12))
sns.catplot(data=dataframe,x='연간소득',y='근로기간')
plt.show()

[1 year과 1 years] level을 보면, 1 years는 항상 1 year 값보다 큰 적이 없다 (즉, 항상 포함된다.)

다른 level 쌍도 동일하게 항상 포함되는 형태를 띄고있음. 

따라서 통합시켜도 될 것 같은 생각이 든다. mean 값도 그렇게 큰 차이를 띄지 않으므로 자료 통합을 할 필요성이 있어보임.

[1 years -> 1 year], [3 -> 3 years], [<1 year -> < 1 year], [10+years -> 10+ years]

In [ ]:
dataframe['근로기간'].value_counts()

# pandas.dataframe.replace(inplace) 에 대한 오류

inplace를 통해서 변경된 내용을 저장하려고했는데, 그렇게 되지 않았다. It doesn't work

inplace와 같은효과를 내려면 대입 연산을 통해서 넣어줘야 한다.

1 years: 56 개 | 1 year: 6249 개  -> 1 year: 6305 개

3: 89 개 | 3 years: 7581 개 -> 3 years: 7670 개

<1 year: 370 개 | < 1 year: 7773 개 -> <1 year: 8143 개

10+years : 896 개 | 10+ years: 31585 개 -> 10+ years: 32481 개로 변경됨

In [ ]:
dataframe['근로기간']=dataframe['근로기간'].replace('1 years','1 year')
dataframe['근로기간']=dataframe['근로기간'].replace('3','3 years')
dataframe['근로기간']=dataframe['근로기간'].replace('<1 year','< 1 year')
dataframe['근로기간']=dataframe['근로기간'].replace('10+years','10+ years')
dataframe['근로기간'].value_counts()

# sns.countplot()으로 범주형 칼럼의 count 개수 시각화

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(data=dataframe,x='근로기간',hue='주택소유상태')
plt.show()